In [10]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [20]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'C:/Users/samsung/ML_Projects/project1/data/train/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Understand the dataset

In [36]:
#Firt let's see the shape of our dataset
print('X shape : ', tX.shape, '||| Y shape : ', y.shape, '||| ids shape : ', ids.shape)

#Let's see what values take y :
print('y takes value in : ',np.unique(y)) #we suppose -1 means that the boson haven't been observed, and 1 mean the opposite

X shape :  (250000, 30) ||| Y shape :  (250000,) ||| ids shape :  (250000,)
y takes value in :  [-1.  1.]


We suppose -1 means that the boson haven't been observed : background, and 1 mean the opposite : signal. Therefore the logistic regression seems to be the most adapted method, as we saw in the course.

Then let's see if the data contains NaN values

In [37]:
print('y has NaN values ?',np.isnan(sum(y)))
print('tX has NaN values ?',np.isnan(sum(sum(tX)))) #tX has 2 dimensions so we sum over the two dimensions
print('ids has NaN values ?', np.isnan(sum(ids)))

y has NaN values ? False
tX has NaN values ? False
ids has NaN values ? False


<ipython-input-37-f0be0e45293c>:3: RuntimeWarning: overflow encountered in long_scalars
  print('ids has NaN values ?', np.isnan(sum(ids)))


# Algorithms :

**1.Least squares with Gradient Descent**

In [118]:
from implementations import *


#Least Squares - GD
max_iters = 10 
gammas = [0.7, 0.1, 1e-5, 1e-6, 5e-7, 3e-7, 1e-7, 1e-8] #by testing the results: this seems to be the optimal value
initial_w = np.zeros((tX.shape[1],)) #number of parameters = number of features of x

for gamma in gammas:
    w_lsGD, loss_lsGD= least_squares_GD(y, tX, initial_w, max_iters, gamma)
    print('for gamma = {}, the loss is : {}'.format(gamma,loss_lsGD))

#We see the best result is for gamma = 3e-7, let's save the loss for this value of gamma and a higher number of iterations
max_iters = 30 
gamma = 3e-7
w_lsGD, loss_lsGD= least_squares_GD(y, tX, initial_w, max_iters, gamma)

for gamma = 0.7, the loss is : 7.205917367667047e+130
for gamma = 0.1, the loss is : 9.030576889717339e+113
for gamma = 1e-05, the loss is : 6.352294868370576e+33
for gamma = 1e-06, the loss is : 1955885360767.4146
for gamma = 5e-07, the loss is : 16217.552088303575
for gamma = 3e-07, the loss is : 0.4253062112549612
for gamma = 1e-07, the loss is : 0.432824131368163
for gamma = 1e-08, the loss is : 0.4569725509407111


Takes a long time to run, let's see the results for gamma = 3e-7

In [119]:
print('loss : {}'.format(loss_lsGD))
print('weights :',w_lsGD)

loss : 0.4162397243972803
weights : [ 3.67792221e-04 -9.67101290e-05 -3.66246121e-05  9.31973035e-06
  1.89374476e-05  1.71686163e-04  1.63451202e-05 -4.69462445e-07
 -2.00833520e-05 -6.34538290e-05 -1.96391984e-06  1.50200694e-06
  1.85905091e-05  2.11952198e-05 -2.24096443e-08 -6.73314380e-08
 -3.07697829e-05 -1.70168484e-08  4.91038347e-08 -1.99995662e-05
  8.88292225e-08 -9.06824590e-05 -9.70587418e-07  4.05086516e-05
  5.09123889e-05  5.09077747e-05 -3.43022887e-06  1.84643781e-05
  1.84259486e-05 -5.38792550e-05]


**2.Least squares with SGD**

In [120]:
max_iters = 10 
gammas = [0.7, 0.1, 1e-5, 3e-7, 1e-7, 1e-8] #by testing the results: this seems to be the optimal value
initial_w = np.ones((tX.shape[1],)) #number of parameters = number of features of x

for gamma in gammas:
    w_lsSGD, loss_lsSGD= least_squares_SGD(y, tX, initial_w, max_iters, gamma)
    print('for gamma = {}, the loss is : {}'.format(gamma,loss_lsSGD))

TypeError: object of type 'numpy.float64' has no len()

**3.Ridge Regression**

In [121]:
lambda_ = 800 #doesn't seem to change anything, we almost sure to be overfitting : loss is very very small
w_rr, loss_rr = ridge_regression(y, tX, lambda_)
print('loss :', loss_rr)
print('weight :', w_rr)

loss : 0.33970212095981306
weight : [ 8.59661758e-05 -7.25581630e-03 -5.91500858e-03 -6.28440493e-04
 -2.06013731e-02  4.76457201e-04 -2.61552746e-02  3.14015392e-01
 -3.94780535e-05  5.29016855e-03 -2.15692112e-01  9.50636706e-02
  6.47124854e-02  2.98414736e-03 -3.34919462e-04 -9.74571843e-04
  7.51105655e-03 -5.29068195e-04  9.64635076e-04  3.74282698e-03
  3.56397337e-04 -5.53042230e-04 -3.17014839e-01 -1.28064587e-03
  7.56716570e-04  9.54285130e-04 -1.60509034e-03 -5.61219034e-03
 -1.07674380e-02 -4.94892129e-03]


**4.Logistic regression**

In [124]:

max_iters = 10 
gammas = [0.7, 0.1, 1e-5, 1e-6, 5e-7, 3e-7, 1e-7, 1e-8] #by testing the results: this seems to be the optimal value
initial_w = np.zeros((tX.shape[1],)) #number of parameters = number of features of x

#Note we use GD !!
for gamma in gammas:
    w_lr, loss_lr= logistic_regression(y, tX, initial_w, max_iters, gamma)
    print('for gamma = {}, the loss is : {}'.format(gamma,loss_lr))

C:\Users\samsung\ML_Projects\project1\scripts\implementations.py:37: RuntimeWarning: overflow encountered in exp
  loss+=np.log(1+np.exp(XiT_W)) - y[i]*XiT_W
C:\Users\samsung\ML_Projects\project1\scripts\implementations.py:28: RuntimeWarning: overflow encountered in exp
  return (np.exp(t)/(1+np.exp(t)))
C:\Users\samsung\ML_Projects\project1\scripts\implementations.py:28: RuntimeWarning: invalid value encountered in true_divide
  return (np.exp(t)/(1+np.exp(t)))


for gamma = 0.7, the loss is : nan
for gamma = 0.1, the loss is : nan
for gamma = 1e-05, the loss is : nan
for gamma = 1e-06, the loss is : nan
for gamma = 5e-07, the loss is : nan
for gamma = 3e-07, the loss is : nan
for gamma = 1e-07, the loss is : nan
for gamma = 1e-08, the loss is : nan


In [126]:
#with regularization :
max_iters = 10 
lambda_ = 1
gammas = [0.7, 0.1, 1e-5, 1e-6, 5e-7, 3e-7, 1e-7, 1e-8] #by testing the results: this seems to be the optimal value
initial_w = np.zeros((tX.shape[1],)) #number of parameters = number of features of x

#Note we use GD !!
for gamma in gammas:
    w_lr, loss_lr= reg_logistic_regression(y, tX, lambda_,initial_w, max_iters, gamma)
    print('for gamma = {}, the loss is : {}'.format(gamma,loss_lr))

for gamma = 0.7, the loss is : nan
for gamma = 0.1, the loss is : nan
for gamma = 1e-05, the loss is : nan
for gamma = 1e-06, the loss is : nan
for gamma = 5e-07, the loss is : nan
for gamma = 3e-07, the loss is : nan
for gamma = 1e-07, the loss is : nan
for gamma = 1e-08, the loss is : nan


## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)